In [46]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

# llm
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI

import os

from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA, ConversationalRetrievalChain

chat_template = """
나는 사용자의 직업을 추천해주는 챗봇이다. 

1. 이력서를 기반으로 저장된 이력서 파일과 유사도를 파악한다. 
2. 파악한 유사도를 기반으로 직업을 3가지 추천해준다. 
3. 이때 추천 직업별로 유사도 퍼센트를 포함하여 제공한다.
4. 직업추천 순서는 유사도가 높은 순서대로 제공한다. 
"""

chat_ai = ChatOpenAI(temperature=0.5, model="gpt-3.5-turbo")

In [ ]:
# Retrieval QA Chain 생성
loader = CSVLoader("../../data/rallit_text.csv", encoding='utf8')
data = loader.load()

# Split the text in chunks, using LangChain Recursive Character Text Splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
    )

pages = loader.load_and_split(text_splitter)

# Create a persistent, file-based vector store, using Chroma vector store.
directory = 'data'
vector_index = Chroma.from_documents(
    pages, # Documents
    OpenAIEmbeddings(), # Text embedding model
    persist_directory=directory # persists the vectors to the file system
    )

vector_index.persist()

# Create the retriever and the query-interface.
retriever = vector_index.as_retriever(
    search_type="similarity", # Cosine Similarity
    search_kwargs={
        "k": 3, # Select top k search results
    }
)

qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0, model="gpt-4"),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True # source document which were used as source files
)


In [6]:
# Retrieval QA Chain 생성
loader = CSVLoader("../../data/rallit_text.csv", encoding='utf8')
docs = loader.load()

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

splits = text_splitter.split_documents(docs)

# 단계 3: 임베딩 & 벡터스토어 생성(Create Vectorstore)
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 단계 4: 검색(Search)
# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

# 단계 5: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = hub.pull("rlm/rag-prompt")
'''
template = """
Answer the question based on the context below. You recommend a user's job.

1. 이력서를 기반으로 저장된 이력서 파일과 유사도를 파악한다. 
2. 파악한 유사도를 기반으로 직업을 3가지 추천해준다. 
3. 이때 추천 직업별로 유사도 퍼센트를 포함하여 제공한다.
4. 직업추천 순서는 유사도가 높은 순서대로 제공한다. 

Context: {context}
Question: {question} 

Answer: """


prompt_template = PromptTemplate(
    input_variables=["question"],
    template=template
)
'''



# 단계 6: 언어모델 생성(Create LLM)
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)


# 단계 7: 체인 생성(Create Chain)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 단계 8: 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "기술스택은 python, sql, java, kubernates이고, 프로젝트는 학교웹사이트를 개발한 적이 있습니다. 데이터베이스 구축과 백엔드 개발을 담당했습니다. 추천 직업을 알려주세요."
#question = "기술스택: python, sql, matplotlib, seaborn, selenium ,프로젝트: 카드사 고객 이탈 데이터 분석을 통한 아이디어 발굴 프로젝트. 크롤링을 통한 데이터 수집과 데이터 시각화, 예측 모델 설계를 담당했습니다. 추천 직업을 알려주세요"
response = rag_chain.invoke(question)

# 결과 출력
print(f"문서의 수: {len(data)}")
print("===" * 20)
print(f"[HUMAN]\n{question}\n")
print(f"[AI]\n{response}")

문서의 수: 705
[HUMAN]
기술스택은 python, sql, java, kubernates이고, 프로젝트는 학교웹사이트를 개발한 적이 있습니다. 데이터베이스 구축과 백엔드 개발을 담당했습니다. 추천 직업을 알려주세요.

[AI]
풀스택 개발자


In [44]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/practice/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=100,
)

folder_path = "../../data/job/"
file_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]

docs = []
for file_path in file_paths:
    loader = CSVLoader(file_path, encoding='utf8')
    docs.extend(loader.load_and_split(text_splitter=splitter))


#loader = CSVLoader("../../data/", encoding='utf8')
#docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever(search_kwargs={'k': 3})

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a helpful job recommendation assistant. 
            Answer the question based on the context below. You recommend a user's job. And just print the job name. Don't print dupliacted job.
            You provide the order of the job recommendations in the order of the highest similarity. 

            1. Determines the similarity of user's resume to a saved resume file. 
            2. It recommends 3 jobs based on the similarity. 

            \n\n
            {context}",
            """
        ),
        ("human", "{question}"),
    ]
)

chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)

result = chain.invoke("기술스택은 python, sql, java, kubernates이고, 프로젝트는 학교웹사이트를 개발한 적이 있습니다. 데이터베이스 구축과 백엔드 개발을 담당했습니다. 추천 직업을 알려주세요.")
print(result)

Created a chunk of size 4361, which is longer than the specified 1000
Created a chunk of size 1108, which is longer than the specified 1000
Created a chunk of size 2631, which is longer than the specified 1000
Created a chunk of size 1232, which is longer than the specified 1000
Created a chunk of size 1288, which is longer than the specified 1000
Created a chunk of size 1003, which is longer than the specified 1000
Created a chunk of size 1017, which is longer than the specified 1000
Created a chunk of size 1179, which is longer than the specified 1000


content='백엔드 개발자' response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 1180, 'total_tokens': 1189}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None} id='run-81f60129-2da3-49a7-89c2-6b4f86924e23-0'


In [11]:
# MMR - 다양성 고려 (lambda_mult = 0.5)
retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 5, 'fetch_k': 50}
)

chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)

result = chain.invoke("기술스택은 python, sql, java, kubernates이고, 프로젝트는 학교웹사이트를 개발한 적이 있습니다. 데이터베이스 구축과 백엔드 개발을 담당했습니다. 추천 직업을 알려주세요.")
print(result)

content='백엔드 개발자' response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 1689, 'total_tokens': 1698}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_a450710239', 'finish_reason': 'stop', 'logprobs': None} id='run-a74b1a1e-df64-4e60-91d4-a9b8e8a56a9e-0'


In [12]:
# Similarity score threshold (기준 스코어 이상인 문서를 대상으로 추출)
retriever = vectorstore.as_retriever(
    search_type='similarity_score_threshold',
    search_kwargs={'score_threshold': 0.3}
)

chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)

result = chain.invoke("기술스택은 python, sql, java, kubernates이고, 프로젝트는 학교웹사이트를 개발한 적이 있습니다. 데이터베이스 구축과 백엔드 개발을 담당했습니다. 추천 직업을 알려주세요.")
print(result)

content='풀스택 개발자' response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 1636, 'total_tokens': 1646}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None} id='run-517b78f3-2fbc-473a-aa01-0d043f8bd6ac-0'


In [8]:
#Refine
retriever = vectorstore.as_retriever()

chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)

result = chain.invoke("기술스택은 python, sql, java, kubernates이고, 프로젝트는 학교웹사이트를 개발한 적이 있습니다. 데이터베이스 구축과 백엔드 개발을 담당했습니다. 추천 직업을 알려주세요.")
print(result)

content='풀스택 개발자' response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 1636, 'total_tokens': 1646}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None} id='run-3f3644ea-a8ef-4f47-8e95-8efc3f4d67d6-0'


In [27]:
template = """
            You are a helpful assistant. 
            Answer the question based on the context below. You recommend a user's job. And just print the job name. Don't print dupliacted job.
            You provide the order of the job recommendations in the order of the highest similarity. 

            1. Determines the similarity of user's resume to a saved resume file. 
            2. It recommends 3 jobs based on the similarity. 

            \n
            {context}
            Question: {question}
            Answer: """


QA_CHAIN_PROMPT = PromptTemplate.from_template(template)# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = vectorstore.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

question = "기술스택은 python, sql, java, kubernates이고, 프로젝트는 학교웹사이트를 개발한 적이 있습니다. 데이터베이스 구축과 백엔드 개발을 담당했습니다. 추천 직업을 알려주세요."
result = qa_chain({"query": question})
# Check the result of the query
result["result"]
# Check the source document from where we 
#result["source_documents"][0]

print(result)

{'query': '기술스택은 python, sql, java, kubernates이고, 프로젝트는 학교웹사이트를 개발한 적이 있습니다. 데이터베이스 구축과 백엔드 개발을 담당했습니다. 추천 직업을 알려주세요.', 'result': '백엔드 개발자', 'source_documents': [Document(page_content='job: 풀스택 개발자\ntext: 기술 스택: Node.js\r\nJava\r\nJavaScript\r\nPython\r\nReact\r\nExpress\r\nPostgreSQL\r\nMySQL\r\nDjango\r\nKotlin\r\nAndroid\r\nGit\r\nGithub\r\n프로젝트 관리\r\nHTML/CSS, 경력: 대진기술정보(주), 연구소장 ? 공간정보연구소, 외주를 통해 S/W 사업을 진행하던 회사에 신규 개발자(연구원)로 커리어를 시작하였으며, 현재까지 독학으로 개발 능력을 키워가며 웹 및 모바일 어플리케이션 개발을 전담하고 있습니다.\r\n기존에 납품하던 로컬 CS 상하수도 관리시스템에 결합할 웹조회시스템을 풀스택으로 개발하고, 유지관리도 담당하고 있습니다. 해당 시스템은 현재 다수의 경북 권내 상하수도 관리기관에 납품되어 있으며, 일선 공무원들이 업무에 가장 많이 활용하는 시스템 중 하나입니다.', metadata={'source': '../../data/rallit_text.csv', 'row': 40}), Document(page_content='job: 백엔드 개발자\ntext: 기술 스택: Python, Django, Java, Spring, Spring Boot, MySQL, PostgreSQL, Docker, AWS, 경력: (사)스마트인재개발원, 연구원 | 기획팀 | 퇴사, 2020.09. ~ 2022.12. (2년 4개월), 주 업무 : Python / Machine Learning 강의\r\n부 업무 : 담임업무 / 강의 스케줄 기획 및 관리, 프로젝트: 주가 이야기, 개인, 2024.02. ~

In [30]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type="map_reduce",
)
result = qa_chain_mr({"query": question})
result["result"]

'주어진 정보에 따라서 사용자에게 추천할 수 있는 직업은 데이터 엔지니어, 백엔드 개발자, 시스템 개발자, 또는 소프트웨어 엔지니어와 같은 직업이 적합할 수 있습니다. 또한, Kubernetes에 대한 지식이 있는 경우 클라우드 엔지니어나 데브옵스 엔지니어로도 진출할 수 있습니다. 이러한 직업들은 주어진 기술 스택과 경험을 활용하여 성장할 수 있는 분야일 수 있습니다.'

In [31]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type="refine"
)
result = qa_chain_mr({"query": question})
result["result"]

'주어진 추가 정보를 바탕으로, 백엔드 개발자로서 Java, Spring, Spring Boot, Node.js와 같은 기술을 사용하여 연합시스템에서 현장실습생으로 일한 경력이 있습니다. 또한, 고등학교에서 도제학교 프로그램에 참여하면서 현업에서의 경험을 쌓았고, 파이썬을 활용하여 자료구조, 알고리즘을 학습하고 GCP MySQL과 연동하여 프로그램을 개발한 경험이 있습니다. MySQL WorkBench를 사용하여 데이터를 검색하고 정상화하는 데 기여한 경험도 있습니다. 이러한 경험을 바탕으로 데이터 엔지니어 또는 백엔드 개발자로서의 경력을 고려할 수 있습니다.'

In [16]:
#Map re-rank
chain = RetrievalQA.from_chain_type(
    llm=llm,
    #prompt=prompt,
    chain_type="map_rerank",
    retriever=retriever,
)

result = chain.invoke("기술스택은 python, sql, java, kubernates이고, 프로젝트는 학교웹사이트를 개발한 적이 있습니다. 데이터베이스 구축과 백엔드 개발을 담당했습니다. 추천 직업을 알려주세요.")
print(result)

c:\Users\hyeon\isic\2024-1-DSCD-ISIC-1\venv\Lib\site-packages\langchain\chains\llm.py:367: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


{'query': '기술스택은 python, sql, java, kubernates이고, 프로젝트는 학교웹사이트를 개발한 적이 있습니다. 데이터베이스 구축과 백엔드 개발을 담당했습니다. 추천 직업을 알려주세요.', 'result': '백엔드 개발자'}


In [9]:
from functools import reduce
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

# 환경 변수를 로드하고 API 키를 설정합니다.
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 언어 모델 초기화
#llm = ChatOpenAI(temperature=0)
llm = ChatOpenAI(model_name="gpt-4", temperature=0)


# 텍스트 분할 설정
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    #separator="\n",
    chunk_size=1000,
    chunk_overlap=100,
)

# 문서 로드 및 분할
folder_path = "../../data/job/"
file_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]


loader = CSVLoader(file_paths, encoding='utf8')
docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
# 벡터 저장소 생성
vectorstore = FAISS.from_documents(docs, embeddings)

# 검색기 설정
retriever = vectorstore.as_retriever(search_kwargs={'k': 3})

# 프롬프트 설정
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a helpful job recommendation assistant. 
            Answer the question based on the context below. You recommend a user's job. And just print the job name. Don't print duplicated jobs.
            You provide the order of the job recommendations in the order of the highest similarity. 

            1. Determines the similarity of the user's resume to a saved resume file. 
            2. It recommends up to 3 jobs based on the similarity. 

            \n\n
            {context}",
            """
        ),
        ("human", "{question}"),
    ]
)

# 실행 체인 정의 및 결과 출력
chain = (
    {
        "context": retriever,
        "question":RunnablePassthrough(),
    }
    | prompt
    | llm
)

result = chain.invoke("기술스택은 python, sql, java, kubernates이고, 프로젝트는 학교웹사이트를 개발한 적이 있습니다. 데이터베이스 구축과 백엔드 개발을 담당했습니다. 추천 직업을 알려주세요.")
print(result)



RuntimeError: Error loading ['../../data/job/output_data_anlysis.csv', '../../data/job/output_data_engineer.csv', '../../data/job/output_devops.csv', '../../data/job/output_pm.csv', '../../data/job/rallit_text.csv']

## Map-reduce

In [2]:
from functools import reduce
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

# 환경 변수를 로드하고 API 키를 설정합니다.
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 언어 모델 초기화
#llm = ChatOpenAI(temperature=0)
llm = ChatOpenAI(model_name="gpt-4", temperature=0)
# 캐시 디렉토리 설정
cache_dir = LocalFileStore("./.cache/practice/")

# 텍스트 분할 설정
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    #separator="\n",
    chunk_size=1000,
    chunk_overlap=100,
)

# 문서 로드 및 분할
folder_path = "../../data/job/"
file_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]

# Map 함수: 각 파일을 로드하고 분할하여 문서 리스트를 반환합니다.
def load_and_split(file_path):
    loader = CSVLoader(file_path, encoding='utf8')
    return loader.load_and_split(text_splitter=splitter)

# Reduce 함수: 여러 문서 리스트를 하나의 리스트로 결합합니다.
def combine_docs(accumulated_docs, docs):
    accumulated_docs.extend(docs)
    return accumulated_docs

# MapReduce 적용: 문서 로드 및 분할, 문서 결합
mapped_docs = map(load_and_split, file_paths)
docs = reduce(combine_docs, mapped_docs, [])

# 임베딩과 캐싱 설정
embeddings = OpenAIEmbeddings()
#cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# 벡터 저장소 생성
vectorstore = FAISS.from_documents(docs, embeddings)

# 검색기 설정
retriever = vectorstore.as_retriever(search_kwargs={'k': 5})

# 프롬프트 설정
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a helpful job recommendation assistant. 
            Answer the question based on the context below. You recommend a user's job. And just print the job name. Don't print duplicated jobs.
            You provide the order of the job recommendations in the order of the highest similarity. 

            1. Determines the similarity of the user's resume to a saved resume file. 
            2. It recommends up to 3 jobs based on the similarity. 

            \n\n
            {context}",
            """
        ),
        ("human", "{question}"),
    ]
)

# 실행 체인 정의 및 결과 출력
chain = (
    {
        "context": retriever,
        "question":RunnablePassthrough(),
    }
    | prompt
    | llm
)

result = chain.invoke("기술스택은 python, sql, java, kubernates이고, 프로젝트는 학교웹사이트를 개발한 적이 있습니다. 데이터베이스 구축과 백엔드 개발을 담당했습니다. 추천 직업을 알려주세요.")
print(result)

content='풀스택 개발자\n백엔드 개발자' response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 2075, 'total_tokens': 2095}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-cccf5280-0341-47f3-89c6-fc08dc7176dd-0'
